In [1]:
import os
import gc
import pickle
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split, StratifiedKFold
from src import WordBaseMLMDataCollator
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm, trange
from sklearn.metrics import accuracy_score
from transformers import get_cosine_schedule_with_warmup, AutoModel, AutoTokenizer, AutoModelForMaskedLM
import wandb
pl.seed_everything(56)

Seed set to 56


56

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
class CFG:
    class data:
        train_path = 'random_walks.pkl'
        tokenizer = 'microsoft/deberta-v3-large'
        num_workers = 8
        nfolds = 5
        batch_size = 64 
        max_length = 70
        mlm_prob = 0.15
        max_mask = 0.5
        min_mask = 1
        seed = 56
        
    class model:
        model = 'microsoft/deberta-v3-large'
        optim=torch.optim.AdamW
        scheduler= 'cosine'
        warnap_steps = 0.1
        label_smoothing = 0.0
        max_epoches = 1
        cls_drop_type = None
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        lr = 4e-6
        turn_off_drop = False
        num_cycles = 0.5
        eps = 1e-7
        weight_decay = 0.0005
        betas = (0.9, 0.999)
    seed = 56
    fold_number = 0

In [4]:
def make_df(path):
    with open(path,'rb') as f:
        walks = pickle.load(f)
    return walks

In [5]:
class PLDataset(Dataset):
    def __init__(self, df, processor):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.processor = processor
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        walk = self.data[index]
        return self.processor(walk)

In [6]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.data
        self.is_setup = False
        self.is_prepared = False
        
    def prepare_data(self):
        if self.is_prepared:
            return None
        self.train_df = make_df(self.cfg.train_path)
        self.tokenizer = AutoTokenizer.from_pretrained(self.cfg.tokenizer)
        self.processor = WordBaseMLMDataCollator(
            tokenizer=self.tokenizer,
            mlm_prob=self.cfg.mlm_prob,
            max_length=self.cfg.max_length,
            min_mask=self.cfg.min_mask,
            max_mask=self.cfg.max_mask
        )
        self.is_prepared = True
        
    def setup(self, stage: str):
        if self.is_setup:
            return None
        self.train_dataset = PLDataset(self.train_df,self.processor)
        self.is_setup = True
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         pin_memory=True,
                         shuffle=True)

In [7]:
class PLModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.model
        self.model = AutoModelForMaskedLM.from_pretrained(self.cfg.model)
        
    def forward(self, batch):
        out = self.model(**batch)
        return out

    def training_step(self, batch, i):
        out = self(batch)
        loss = out.loss
        self.log('train_loss', loss.item())
        
        if self.global_step % 5_000 == 100:
            self.trainer.save_checkpoint(f'./outputs/chekpoint_{i}.ckpt')
        return loss
                                    
    def configure_optimizers(self):
        optimizer_parameters = [
            {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': self.cfg.weight_decay},
            {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': 0.0},
        ]
        
        optim = self.cfg.optim(
            optimizer_parameters,
            lr=self.cfg.lr,
            betas=self.cfg.betas,
            weight_decay=self.cfg.weight_decay,
            eps=self.cfg.eps
        )
        
        if self.cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warnap_steps,
                                                        num_cycles=self.cfg.num_cycles)
        elif self.cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warnap_steps)
        else:
            return optim
        
        scheduler = {'scheduler': scheduler,'interval': 'step', 'frequency': 1}

        return [optim], [scheduler]

In [8]:
dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
CFG.model.num_training_steps = (len(dm.train_dataloader())) * CFG.model.max_epoches

In [10]:
model = PLModule()

Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
CFG.model.num_training_steps

4719

In [12]:
wandb.login(key="31520b01739d418e5d77a11fd8a79a70b189b8bc")
os.environ['WANDB_API_KEY'] = "31520b01739d418e5d77a11fd8a79a70b189b8bc"
wandb.init(project='AIIJC_task1',name='deberta_V3_large_mlm_loe')

wandb: Currently logged in as: andrewkhl (andlh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [16]:
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
checkpoint_cb = pl.callbacks.ModelCheckpoint(
    dirpath='./outputs/',
    filename='model',
    #monitor='accuracy',
    #mode='max',
    save_last=True
)

trainer = pl.Trainer(
    accelerator="gpu",
    precision=32,
    callbacks = [lr_monitor,checkpoint_cb],
    logger = pl.loggers.WandbLogger(save_code=True),
    log_every_n_steps=1,
    min_epochs=1,
    val_check_interval=None,
    devices=1,
    check_val_every_n_epoch=1,
    max_epochs=CFG.model.max_epoches
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                 | Params | Mode
------------------------------------------------------
0 | model | DebertaV2ForMaskedLM | 435 M  | eval
------------------------------------------------------
435 M     Trainable params
0         Non-trainable params
435 M     Total params
1,740.768 Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [19]:
torch.save(model.state_dict(),'deberta_smallmlm.pt')